In [1]:
def encoder_block(inputs, num_filters): 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding='same')(inputs) 
    x = tf.keras.layers.Activation('relu')(x) 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding='same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
    x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x) 
    return x


In [2]:
import tensorflow as tf

def decoder_block(inputs, skip_features, num_filters): 
    # Upsample the input tensor
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs) 

    # Resize skip_features to match the spatial dimensions of x using Lambda
    skip_features_resized = tf.keras.layers.Lambda(
        lambda y: tf.image.resize(y, size=(x.shape[1], x.shape[2]))  # Using x.shape after upscaling
    )(skip_features)

    # Concatenate the skip connection
    x = tf.keras.layers.Concatenate()([x, skip_features_resized]) 

    # Apply two convolutional layers
    x = tf.keras.layers.Conv2D(num_filters, 3, padding='same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding='same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
    
    return x


2024-11-25 19:29:59.357311: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 19:29:59.378026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732541399.403946  178929 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732541399.409207  178929 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 19:29:59.436785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
def unet_model(input_shape=(256, 256, 3), num_classes=1): 
    inputs = tf.keras.layers.Input(input_shape) 

    # Contracting Path 
    s1 = encoder_block(inputs, 64) 
    s2 = encoder_block(s1, 128) 
    s3 = encoder_block(s2, 256) 
    s4 = encoder_block(s3, 512) 

    # Bottleneck 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding='same')(s4) 
    b1 = tf.keras.layers.Activation('relu')(b1) 
    b1 = tf.keras.layers.Conv2D(1024, 3, padding='same')(b1) 
    b1 = tf.keras.layers.Activation('relu')(b1) 

    # Expansive Path 
    s5 = decoder_block(b1, s4, 512) 
    s6 = decoder_block(s5, s3, 256) 
    s7 = decoder_block(s6, s2, 128) 
    s8 = decoder_block(s7, s1, 64) 

    # Output Layer 
    outputs = tf.keras.layers.Conv2D(num_classes, 1, activation='sigmoid' if num_classes == 1 else 'softmax')(s8) 

    model = tf.keras.models.Model(inputs, outputs, name='U-Net') 
    return model


In [7]:
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

img_size = (256, 256)

# Load and preprocess the image
img = Image.open('car.jpg').resize(img_size)  # Match model input
img_array = img_to_array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array[:,:,:3], axis=0)  # Add batch dimension

# Load the model
model = unet_model(input_shape=(256, 256, 3), num_classes=2)

# Predict
predictions = model.predict(img_array)

# Postprocess predictions
predictions = np.squeeze(predictions, axis=0)
if predictions.shape[-1] == 1:  # Binary case
    predictions = (predictions > 0.5).astype(np.uint8)
else:  # Multi-class case
    predictions = np.argmax(predictions, axis=-1)

# Save the predicted image
output = Image.fromarray(predictions.astype(np.uint8) * 255)
output.save('predicted_image.jpg')

out = Image.open('predicted_image.jpg')

img.show()
out.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Opening in existing browser session.
Opening in existing browser session.


Error: ../src/amd/vulkan/radv_physical_device.c:1813: Could not open device /dev/dri/renderD128: Permission denied (VK_ERROR_INCOMPATIBLE_DRIVER)
Error: ../src/amd/vulkan/radv_physical_device.c:1813: Could not open device /dev/dri/renderD128: Permission denied (VK_ERROR_INCOMPATIBLE_DRIVER)


In [50]:
import tensorflow as tf

def conv_block(inputs, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)

    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = tf.keras.layers.MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip, num_filters):
    # Apply transposed convolution to upsample the input
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)

    # Resize the skip connection using tf.image.resize in a Lambda layer
    def resize_skip(skip):
        return tf.image.resize(skip, size=tf.shape(x)[1:3])

    skip_resized = tf.keras.layers.Lambda(resize_skip, output_shape=lambda input_shape: (input_shape[0], x.shape[1], x.shape[2], input_shape[3]))(skip)

    # Concatenate the upsampled skip connection with the upsampled input
    x = tf.keras.layers.Concatenate()([x, skip_resized])

    # Apply another convolution block
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = tf.keras.layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)  # 500 x 500
    s2, p2 = encoder_block(p1, 128)  # 250 x 250
    s3, p3 = encoder_block(p2, 256)  # 125 x 125
    s4, p4 = encoder_block(p3, 512)  # 62 x 62

    # Bridge
    b1 = conv_block(p4, 1024)  # 31 x 31

    # Decoder
    d1 = decoder_block(b1, s4, 512)  # 62 x 62
    d2 = decoder_block(d1, s3, 256)  # 125 x 125
    d3 = decoder_block(d2, s2, 128)  # 250 x 250
    d4 = decoder_block(d3, s1, 64)   # 500 x 500

    # Output layer
    outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = tf.keras.models.Model(inputs, outputs, name="UNET")
    return model

# Example usage:
input_shape = (250, 250, 3)  # Adjust as needed
model = build_unet(input_shape)
model.summary()


Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_276 (Conv2D) │ (None, 250, 250,  │      1,792 │ input_layer_25[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 250,  │        256 │ conv2d_276[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_176     │ (None, 250, 250,  │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_277 (Conv2D) │ (None, 250, 250,  │     36,928 │ leaky_re_lu_176[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 250,  │        256 │ conv2d_277[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_177     │ (None, 250, 250,  │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_96    │ (None, 125, 125,  │          0 │ leaky_re_lu_177[… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_278 (Conv2D) │ (None, 125, 125,  │     73,856 │ max_pooling2d_96… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │        512 │ conv2d_278[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_178     │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_279 (Conv2D) │ (None, 125, 125,  │    147,584 │ leaky_re_lu_178[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │        512 │ conv2d_279[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_179     │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_97    │ (None, 62, 62,    │          0 │ leaky_re_lu_179[… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_280 (Conv2D) │ (None, 62, 62,    │    295,168 │ max_pooling2d_97… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 62, 62,    │      1,024 │ conv2d_280[0][0]

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)